In [2]:
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import string
import re
import string
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

from datetime import datetime as dt
import numpy as np
from functools import reduce

#from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler

from matplotlib.colors import LogNorm
%matplotlib inline

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

import scipy as sp
from scipy.stats import multivariate_normal    

import imageio
import sys
np.set_printoptions(threshold=sys.maxsize)

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, e

In [5]:
data = pd.read_pickle('preprocessed_data.pkl')
data.index

RangeIndex(start=0, stop=35728, step=1)

In [6]:
print(data.columns.tolist())

['YEAR', 'ITERATION', 'ID_HH', 'RHoMIS_ID', 'GPS_LAT', 'GPS_LON', 'GPS_ALT', 'Altitude', 'HHsizemembers', 'HHsizeMAE', 'Head_EducationLevel', 'LandOwned', 'LandCultivated', 'LivestockHoldings', 'WorstFoodSecMonth', 'BestFoodSecMonth', 'NrofMonthsFoodInsecure', 'PPI_Threshold', 'PPI_Likelihood', 'HFIAS_status', 'FIES_Score', 'score_HDDS_GoodSeason', 'score_HDDS_farmbasedGoodSeason', 'score_HDDS_purchasedGoodSeason', 'score_HDDS_BadSeason', 'score_HDDS_farmbasedBadSeason', 'score_HDDS_purchasedBadSeason', 'TVA_USD_PPP_pmae_pday', 'currency_conversion_factor', 'total_income_USD_PPP_pHH_Yr', 'offfarm_income_USD_PPP_pHH_Yr', 'farm_income_USD_PPP_pHH_Yr', 'value_farm_produce_USD_PPP_pHH_Yr', 'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr', 'value_crop_consumed_USD_PPP_pHH_Yr', 'livestock_prodsales_USD_PPP_pHH_Yr', 'value_livestock_production_USD_PPP_pHH_Yr', 'value_livestock_prod_consumed_USD_PPP_pHH_Yr', 'Market_Orientation', 'Livestock_Orientation', 'Food_Availability_kCal

In [7]:
data_pre = data[['GPS_LAT', 'GPS_ALT', 'LandOwned', 'LandCultivated', 'LivestockHoldings', 'Food_Availability_kCal_MAE_day', 'Food_Self_Sufficiency_kCal_MAE_day',
'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr', 'GHGEmissions']]

In [8]:
data_pre

,GPS_LAT,GPS_ALT,LandOwned,LandCultivated,LivestockHoldings,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,crop_sales_USD_PPP_pHH_Yr,value_crop_produce_USD_PPP_pHH_Yr,GHGEmissions
0,11.19,254.00,10.0,2.0,1.04,4621.267353,754.360895,164.857702,242.318219,702.949550
1,11.19,346.00,NaN,1.2,4.00,2503.023263,2303.522017,0.000000,279.844923,94415.087935
2,11.19,324.00,2.0,0.8,0.30,381.108158,381.108158,0.000000,23.278518,113.654565
3,11.19,326.00,NaN,0.8,5.43,9948.141738,620.787518,228.969031,255.682085,43062.200002
4,11.22,308.00,10.0,4.0,2.12,2209.261281,1767.543491,22.896903,184.734710,13331.792068
...,...,...,...,...,...,...,...,...,...,...
35723,-13.82,1119.60,8.0,8.0,3.58,21491.305517,12770.090428,2970.040440,9187.374358,34884.136689
35724,-13.82,1128.85,NaN,14.0,8.80,11260.499490,4217.809731,4501.379943,8866.040295,79114.978038
35725,-13.82,1132.41,5.0,6.0,5.48,4247.172555,1221.939610,1083.897904,2007.669471,46350.365405
35726,-13.82,1136.24,4.0,4.0,0.70,5918.719620,5918.719620,0.000000,2026.134892,7452.589981


In [9]:
food_available = data_pre[['Food_Availability_kCal_MAE_day']]
# food_available

In [10]:
livestock = data_pre[['LivestockHoldings']]
# livestock

In [11]:
crop = data_pre[['value_crop_produce_USD_PPP_pHH_Yr']]
# crop

In [12]:
GHC = data_pre[['GHGEmissions']]
LAT = data_pre[['GPS_LAT']]
ALT = data_pre[['GPS_ALT']]
land = data_pre[['LandOwned']]  
land_cult = data_pre[['LandCultivated']] 
food_self = data_pre[['Food_Self_Sufficiency_kCal_MAE_day']]
val_crop = data_pre[['value_crop_produce_USD_PPP_pHH_Yr']]

In [13]:
food_available.isnull().values.any()
food_clean = food_available.dropna()
food_available_nans = food_available[food_available['Food_Availability_kCal_MAE_day'].isnull()]
# print(food_clean)
# food_available_nans

In [14]:
food1 = food_available_nans.to_numpy()
food2 = food_clean.to_numpy()
# food_pre = np.array(food2.tolist(), food1.tolist())
# food_pre

In [15]:
foodie_pre = food_available.to_numpy()
livie_pre = livestock.to_numpy()
crop_pre = crop.to_numpy()
GHC_pre = GHC.to_numpy()
LAT_pre = LAT.to_numpy()
ALT_pre = ALT.to_numpy()
land_pre = land.to_numpy()
food_self_pre = food_self.to_numpy()
val_crop_pre = val_crop.to_numpy()
land_cult_pre = land_cult.to_numpy()
# foodie_pre = np.array(food_available)
print(len(foodie_pre), len(livie_pre), len(crop_pre), len(GHC_pre))
print(foodie_pre.shape, livie_pre.shape, crop_pre.shape, GHC_pre.shape)
# food2

35728 35728 35728 35728
(35728, 1) (35728, 1) (35728, 1) (35728, 1)


In [16]:
food_list = []
for i in range(len(foodie_pre)):
    food_list.append(np.asscalar(foodie_pre[i]))
    
live_list = []
for i in range(len(livie_pre)):
    live_list.append(np.asscalar(livie_pre[i]))

crop_list = []
for i in range(len(crop_pre)):
    crop_list.append(np.asscalar(crop_pre[i]))

GHC_list = []
for i in range(len(GHC_pre)):
    GHC_list.append(np.asscalar(GHC_pre[i]))
    
LAT_list = []
for i in range(len(GHC_pre)):
    LAT_list.append(np.asscalar(LAT_pre[i]))

ALT_list = []
for i in range(len(GHC_pre)):
    ALT_list.append(np.asscalar(ALT_pre[i]))

land_list = []
for i in range(len(GHC_pre)):
    land_list.append(np.asscalar(land_pre[i]))

food_self_list = []
for i in range(len(GHC_pre)):
    food_self_list.append(np.asscalar(food_self_pre[i]))

val_crop_list = []
for i in range(len(GHC_pre)):
    val_crop_list.append(np.asscalar(val_crop_pre[i]))

land_cult_list = []
for i in range(len(GHC_pre)):
    land_cult_list.append(np.asscalar(land_cult_pre[i]))

print(len(live_list), len(food_list), len(crop_list), len(GHC_list))

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  import sys
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  from ipykernel import kernelapp as app
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
C:\Users\Asus\Anaconda3

35728 35728 35728 35728


C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


In [17]:
predict = np.array(list(zip(live_list, food_list, land_list)))
# predict

In [18]:
def impute_em(X, max_iter = 3000, eps = 1e-08):
    '''(np.array, int, number) -> {str: np.array or int}
    
    Precondition: max_iter >= 1 and eps > 0
    
    Return the dictionary with five keys where:
    - Key 'mu' stores the mean estimate of the imputed data.
    - Key 'Sigma' stores the variance estimate of the imputed data.
    - Key 'X_imputed' stores the imputed data that is mutated from X using 
      the EM algorithm.
    - Key 'C' stores the np.array that specifies the original missing entries
      of X.
    - Key 'iteration' stores the number of iteration used to compute
      'X_imputed' based on max_iter and eps specified.
    '''
    
    nr, nc = X.shape
    C = np.isnan(X) == False
    
    # Collect M_i and O_i's
    one_to_nc = np.arange(1, nc + 1, step = 1)
    M = one_to_nc * (C == False) - 1
    O = one_to_nc * C - 1
    
    # Generate Mu_0 and Sigma_0
    Mu = np.nanmean(X, axis = 0)
    observed_rows = np.where(np.isnan(sum(X.T)) == False)[0]
    S = np.cov(X[observed_rows, ].T)
    if np.isnan(S).any():
        S = np.diag(np.nanvar(X, axis = 0))
    
    # Start updating
    Mu_tilde, S_tilde = {}, {}
    X_tilde = X.copy()
    no_conv = True
    iteration = 0
    while no_conv and iteration < max_iter:
        for i in range(nr):
            S_tilde[i] = np.zeros(nc ** 2).reshape(nc, nc)
            if set(O[i, ]) != set(one_to_nc - 1): # missing component exists
                M_i, O_i = M[i, ][M[i, ] != -1], O[i, ][O[i, ] != -1]
                S_MM = S[np.ix_(M_i, M_i)]
                S_MO = S[np.ix_(M_i, O_i)]
                S_OM = S_MO.T
                S_OO = S[np.ix_(O_i, O_i)]
                Mu_tilde[i] = Mu[np.ix_(M_i)] +\
                    S_MO @ np.linalg.inv(S_OO) @\
                    (X_tilde[i, O_i] - Mu[np.ix_(O_i)])
                X_tilde[i, M_i] = Mu_tilde[i]
                S_MM_O = S_MM - S_MO @ np.linalg.inv(S_OO) @ S_OM
                S_tilde[i][np.ix_(M_i, M_i)] = S_MM_O
        Mu_new = np.mean(X_tilde, axis = 0)
        S_new = np.cov(X_tilde.T, bias = 1) +\
            reduce(np.add, S_tilde.values()) / nr
        no_conv =\
            np.linalg.norm(Mu - Mu_new) >= eps or\
            np.linalg.norm(S - S_new, ord = 2) >= eps
        Mu = Mu_new
        S = S_new
        iteration += 1
    
    result = {
        'mu': Mu,
        'Sigma': S,
        'X_imputed': X_tilde,
        'C': C,
        'iteration': iteration
    }
    
    return result

In [19]:
start = dt.now()
result_imputed = impute_em(predict)
end = dt.now()

In [20]:
# result_imputed

In [21]:
result_imputed['mu']

array([1.49159836e+01, 1.88741048e+07, 1.32369781e+01])

In [22]:
# mu

In [23]:
result_imputed['Sigma']

array([[ 4.10914734e+06, -2.51662473e+08,  2.58177489e+00],
       [-2.51662473e+08,  2.93678998e+18, -8.94313969e+07],
       [ 2.58177489e+00, -8.94313969e+07,  8.81106437e+04]])

In [24]:
# Sigma

In [25]:
np.sum(predict)

nan

In [26]:
np.sum(result_imputed['X_imputed'])

674335023804.2489

In [27]:
# predict

In [55]:
array = result_imputed['X_imputed']
print((array >= 0).all())
array[:,2]




False


array([1.00000000e+01, 1.32375460e+01, 2.00000000e+00, 1.32375466e+01,
       1.00000000e+01, 2.00000000e+01, 4.00000000e+00, 2.00000000e+00,
       8.00000000e-01, 4.00000000e-01, 2.00000000e+00, 1.20000000e+00,
       4.00000000e-01, 1.60000000e+00, 4.00000000e-01, 1.20000000e+00,
       8.00000000e-01, 8.00000000e-01, 1.20000000e+00, 4.00000000e+00,
       8.00000000e-01, 2.40000000e+00, 4.00000000e+00, 1.32375471e+01,
       1.32375451e+01, 3.20000000e+00, 2.40000000e+00, 2.00000000e+00,
       3.60000000e+00, 1.32375435e+01, 3.00000000e+00, 1.32375447e+01,
       1.20000000e+00, 4.00000000e-01, 1.32375470e+01, 1.32375467e+01,
       1.32375505e+01, 1.32375466e+01, 0.00000000e+00, 1.60000000e+00,
       2.00000000e+00, 2.80000000e+00, 1.60000000e+00, 1.32375481e+01,
       1.20000000e+00, 2.40000000e+00, 4.00000000e-01, 0.00000000e+00,
       1.32375456e+01, 1.60000000e+00, 0.00000000e+00, 2.40000000e+00,
       6.80000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [29]:
# class GMM(object):
#     def __init__(self, X, k=2):
#         # dimension
#         X = np.asarray(X)
#         self.m, self.n = X.shape
#         self.data = X.copy()
#         # number of mixtures
#         self.k = k
        
#     def _init(self):
#         # init mixture means/sigmas
#         self.mean_arr = np.asmatrix(np.random.random((self.k, self.n)))
#         self.sigma_arr = np.array([np.asmatrix(np.identity(self.n)) for i in range(self.k)])
#         self.phi = np.ones(self.k)/self.k
#         self.w = np.asmatrix(np.empty((self.m, self.k), dtype=float))
#         #print(self.mean_arr)
#         #print(self.sigma_arr)
    
#     def fit(self, tol=1e-4):
#         self._init()
#         num_iters = 0
#         ll = 1
#         previous_ll = 0
#         while(ll-previous_ll > tol):
#             previous_ll = self.loglikelihood()
#             self._fit()
#             num_iters += 1
#             ll = self.loglikelihood()
#             print('Iteration %d: log-likelihood is %.6f'%(num_iters, ll))
#         print('Terminate at %d-th iteration:log-likelihood is %.6f'%(num_iters, ll))
    
#     def loglikelihood(self):
#         ll = 0
#         for i in range(self.m):
#             tmp = 0
#             for j in range(self.k):
#                 #print(self.sigma_arr[j])
#                 tmp += sp.stats.multivariate_normal.pdf(self.data[i, :], 
#                                                         self.mean_arr[j, :].A1, 
#                                                         self.sigma_arr[j, :]) *\
#                        self.phi[j]
#             ll += np.log(tmp) 
#         return ll
    
#     def _fit(self):
#         self.e_step()
#         self.m_step()
        
#     def e_step(self):
#         # calculate w_j^{(i)}
#         for i in range(self.m):
#             den = 0
#             for j in range(self.k):
#                 num = sp.stats.multivariate_normal.pdf(self.data[i, :], 
#                                                        self.mean_arr[j].A1, 
#                                                        self.sigma_arr[j]) *\
#                       self.phi[j]
#                 den += num
#                 self.w[i, j] = num
#             self.w[i, :] /= den
#             assert self.w[i, :].sum() - 1 < 1e-4
            
#     def m_step(self):
#         for j in range(self.k):
#             const = self.w[:, j].sum()
#             self.phi[j] = 1/self.m * const
#             _mu_j = np.zeros(self.n)
#             _sigma_j = np.zeros((self.n, self.n))
#             for i in range(self.m):
#                 _mu_j += (self.data[i, :] * self.w[i, j])
#                 _sigma_j += self.w[i, j] * ((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
#                 #print((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
#             self.mean_arr[j] = _mu_j / const
#             self.sigma_arr[j] = _sigma_j / const
#         #print(self.sigma_arr)

In [30]:
# gmm = GMM(food2)
# gmm.fit()

In [31]:
X = np.random.multivariate_normal([0, 3], [[0.5, 0], [0, 0.8]], 20)
X = np.vstack((X, np.random.multivariate_normal([20, 10], np.identity(2), 50)))
X.shape

(70, 2)

In [32]:
np.random.seed(1024)
mu = np.array([1, 2, 6])
Sigma = np.array([[118, 62, 44], [62, 49, 17], [44, 17, 21]])
n = 400
X_truth = np.random.multivariate_normal(mu, Sigma, n)

In [33]:
X_truth

array([[-2.25150430e+01, -1.12193154e+01, -3.16804411e-01],
       [-4.13801604e+00, -3.38133110e+00,  3.97850866e+00],
       [-6.96117925e+00, -4.78845229e+00,  1.21653198e+00],
       [ 6.57201047e+00,  6.05202260e+00,  8.87408451e+00],
       [ 8.07906102e+00,  3.46018924e+00,  9.17663177e+00],
       [-2.71777693e+00,  1.51178112e+00,  4.17435875e+00],
       [-5.25896933e+00, -5.44848469e+00,  4.44208904e+00],
       [-4.17891721e+00, -3.48215875e+00,  4.29849034e+00],
       [-1.12964709e+01,  1.28621915e+00,  9.84027061e-01],
       [ 4.77382121e+00,  6.22432395e+00,  7.37910714e+00],
       [ 1.76714708e+01,  1.48482915e+01,  1.06402793e+01],
       [-2.62746686e+00, -2.83818082e+00,  5.32422802e+00],
       [ 1.41467046e+00,  1.48388607e+00,  4.71671684e+00],
       [-4.51887281e+00, -3.36943171e+00,  6.80661022e+00],
       [ 1.42295731e+01,  8.98458305e+00,  1.30355167e+01],
       [ 4.56520129e+00,  1.58082626e+00,  9.90645084e+00],
       [-3.10142281e+01, -1.81147725e+01

In [34]:
def simulate_nan(X, nan_rate):
    '''(np.array, number) -> {str: np.array or number}
    
    Preconditions:
    1. np.isnan(X_complete).any() == False
    2. 0 <= nan_rate <= 1
    
    Return the dictionary with four keys where: 
    - Key 'X' stores a np.array where some of the entries in X 
      are replaced with np.nan based on nan_rate specified.
    - Key 'C' stores a np.array where each entry is False if the
      corresponding entry in the key 'X''s np.array is np.nan, and True
      otherwise.
    - Key 'nan_rate' stores nan_rate specified.
    - Key 'nan_rate_actual' stores the actual proportion of np.nan
      in the key 'X''s np.array.
    '''
    
    # Create C matrix; entry is False if missing, and True if observed
    X_complete = X.copy()
    nr, nc = X_complete.shape
    C = np.random.random(nr * nc).reshape(nr, nc) > nan_rate
    
    # Check for which i's we have all components become missing
    checker = np.where(sum(C.T) == 0)[0]
    if len(checker) == 0:
        # Every X_i has at least one component that is observed,
        # which is what we want
        X_complete[C == False] = np.nan
    else:
        # Otherwise, randomly "revive" some components in such X_i's
        for index in checker:
            reviving_components = np.random.choice(
                nc, 
                int(np.ceil(nc * np.random.random())), 
                replace = False
            )
            C[index, np.ix_(reviving_components)] = True
        X_complete[C == False] = np.nan
    
    result = {
        'X': X_complete,
        'C': C,
        'nan_rate': nan_rate,
        'nan_rate_actual': np.sum(C == False) / (nr * nc)
    }
    
    return result

In [35]:
result = simulate_nan(X_truth, nan_rate = .4)
X = result['X'].copy()
X

array([[-2.25150430e+01,             nan,             nan],
       [-4.13801604e+00,             nan,             nan],
       [-6.96117925e+00, -4.78845229e+00,  1.21653198e+00],
       [ 6.57201047e+00,  6.05202260e+00,  8.87408451e+00],
       [            nan,             nan,  9.17663177e+00],
       [            nan,  1.51178112e+00,             nan],
       [            nan, -5.44848469e+00,  4.44208904e+00],
       [-4.17891721e+00, -3.48215875e+00,  4.29849034e+00],
       [-1.12964709e+01,  1.28621915e+00,             nan],
       [ 4.77382121e+00,  6.22432395e+00,  7.37910714e+00],
       [ 1.76714708e+01,             nan,  1.06402793e+01],
       [            nan, -2.83818082e+00,  5.32422802e+00],
       [ 1.41467046e+00,  1.48388607e+00,             nan],
       [-4.51887281e+00, -3.36943171e+00,  6.80661022e+00],
       [ 1.42295731e+01,  8.98458305e+00,             nan],
       [ 4.56520129e+00,  1.58082626e+00,  9.90645084e+00],
       [-3.10142281e+01,             nan

In [36]:
(sum((np.isnan(X) == False).T) == 0).any() == False

True

In [37]:
result['nan_rate_actual']

0.3933333333333333